In [1]:
import torch
# from transformers import OPTForCausalLM, AutoModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
# from chemlactica.utils.utils import get_tokenizer
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from rdkit.Chem import RDConfig, QED

from rdkit.Chem.rdMolDescriptors import CalcTPSA

/auto/home/menuab/miniforge3/envs/gemma_env_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained("/auto/home/menuab/code/ChemLactica/chemlactica/tokenizer/GemmaTokenizer")

In [4]:
model_path = "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/52924785fbfc4c2e839d7e43/2b_11k_5292"
# model_path = "/auto/home/menuab/code/checkpoints/26d322857a184fcbafda5d4a/125m_118k_26d3/"
# OPTForCausalLM.__init__ = cast_lm_head_to_fp32_init(OPTForCausalLM.__init__)
model = AutoModelForCausalLM.from_pretrained(model_path).eval().to('cuda')
# model = 

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:52<00:00, 26.44s/it]


In [65]:
tpsa_score = AllChem.CalcTPSA(Chem.MolFromSmiles("CC(C)(C)OC(=O)N1CCCC1C(=O)N1CCCC1C(=O)NC(Cc1ccc(O)cc1)C(=O)O"))

In [74]:
CalcTPSA(Chem.MolFromSmiles("CC(C)(C)OC(=O)N1CCCC1C(=O)N1CCCC1C(=O)NC(Cc1ccc(O)cc1)C(=O)O")), tpsa_score

(136.48, 136.48)

In [10]:
prompt = f"[TPSA]1.4[/TPSA]"
len_prompt = len(prompt)
prompt = tokenizer(prompt, return_tensors="pt").to(model.device)
out = model.generate(prompt.input_ids, do_sample=False, eos_token_id=20, max_length=300)
out = tokenizer.batch_decode(out)[0]

In [11]:
out

'[TPSA]1.4[/TPSA][NUMAROMATICRINGS]1[/NUMAROMATICRINGS][NHOHCOUNT]1[/NHOHCOUNT][NUMALIPHATICRINGS]1[/NUMALIPHATICRINGS][NUMALIPHATICHETEROCYCLES]1[/NUMALIPHATICHETEROCYCLES][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][NUMAROMATICCARBOCYCLES]1[/NUMAROMATICCARBOCYCLES][FORMULA]C15H20N2O2[/FORMULA][NUMSATURATEDRINGS]1[/NUMSATURATEDRINGS][RINGCOUNT]2[/RINGCOUNT][CLOGP]2.28[/CLOGP][NUMHETEROATOMS]4[/NUMHETEROATOMS][NOCOUNT]4[/NOCOUNT][FRACTIONCSP3]0.47[/FRACTIONCSP3][NUMROTATABLEBONDS]4[/NUMROTATABLEBONDS][IUPAC]2-[(2-methyl-3-phenylpropanoyl)amino]piperidine-1-carbonitrile[/IUPAC][HEAVYATOMCOUNT]19[/HEAVYATOMCOUNT][NUMHACCEPTORS]'

In [13]:
ground_truths, gens, diffs = [],[],[]
invalids = 0
for i in np.arange(0,100, 0.2):
    prompt = f"<bos>[TPSA]{i:.2f}[/TPSA]"
    len_prompt = len(prompt)
    prompt = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(prompt.input_ids, do_sample=False, eos_token_id=20, max_length=300)
    out = tokenizer.batch_decode(out)[0]
    try:
        if out.find("[END_SMILES]")!=-1:
            smiles = out[out.find("[START_SMILES]") + len("[START_SMILES]"):out.find("[END_SMILES]")]
            tpsa_score = AllChem.CalcTPSA(Chem.MolFromSmiles(smiles))
            diff = abs(i - tpsa_score)
            print("GT:", i, "Gen:", tpsa_score, "diff:", round(diff,2), smiles, out )
            ground_truths.append(i)
            gens.append(tpsa_score)
            diffs.append(diff)
        else:
            print(f"GT: {i} {out}")
    except:
        print(f"GT: {i} {out}")
        invalids += 1
        pass

GT: 0.0 Gen: 0.0 diff: 0.0 CC(C)C(C)C(C)C(C)C=S <bos>[TPSA]0.00[/TPSA][NUMAROMATICRINGS]0[/NUMAROMATICRINGS][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][NHOHCOUNT]0[/NHOHCOUNT][CLOGP]4.36[/CLOGP][RINGCOUNT]0[/RINGCOUNT][NUMHDONORS]0[/NUMHDONORS][NUMROTATABLEBONDS]1[/NUMROTATABLEBONDS][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][FORMULA]C10H20S[/FORMULA][FRACTIONCSP3]0.80[/FRACTIONCSP3][NUMALIPHATICRINGS]0[/NUMALIPHATICRINGS][NOCOUNT]0[/NOCOUNT][NUMSATURATEDHETEROCYCLES]0[/NUMSATURATEDHETEROCYCLES][START_SMILES]CC(C)C(C)C(C)C(C)C=S[END_SMILES][IUPAC]3,4,5,6-tetramethylheptane-1-thial[/IUPAC][NUMHETEROATOMS]1[/NUMHETEROATOMS][NUMAROMATICCARBOCYCLES]0[/NUMAROMATICCARBOCYCLES][NUMHACCEPTORS]
GT: 0.2 Gen: 0.0 diff: 0.2 CCCC(C)(SC)C(C)(S)C(C)C <bos>[TPSA]0.20[/TPSA][NUMAROMATICRINGS]0[/NUMAROMATICRINGS][NHOHCOUNT]0[/NHOHCOUNT][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][CLOGP]4.36[/CLOGP][RINGCOUNT]0[/RINGCOUNT][NU

[13:13:15] WARNING: not removing hydrogen atom without neighbors


GT: 19.6 Gen: 40.32 diff: 20.72 Cc1ccc(N=CC=NN2C=NCC2)cc1.[HH] <bos>[TPSA]19.60[/TPSA][NHOHCOUNT]0[/NHOHCOUNT][NUMAROMATICRINGS]1[/NUMAROMATICRINGS][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][NUMSATURATEDHETEROCYCLES]0[/NUMSATURATEDHETEROCYCLES][WEIGHT]260.16[/WEIGHT][RINGCOUNT]2[/RINGCOUNT][NUMALIPHATICRINGS]1[/NUMALIPHATICRINGS][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][NUMHDONORS]0[/NUMHDONORS][NUMALIPHATICHETEROCYCLES]1[/NUMALIPHATICHETEROCYCLES][FORMULA]C15H20N4[/FORMULA][IUPAC]1-[(E)-2-(4-methylphenyl)iminoethylideneamino]-4,5-dihydroimidazole;molecular hydrogen[/IUPAC][CLOGP]2.56[/CLOGP][NUMROTATABLEBONDS]3[/NUMROTATABLEBONDS][START_SMILES]Cc1ccc(N=CC=NN2C=NCC2)cc1.[HH][END_SMILES][NOCOUNT]4[/NOCOUNT][NUMHETEROATOMS]4[/NUMHETEROATOMS][NUMAROMATICCARBOCYCLES]1[/NUMAROMATICCARBOCYCLES][FRACTIONCSP3]0.27[/FRACTIONCSP3][QED]0.75[/QED][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][NUMSATURATEDRINGS]0[/NUMSATURATEDRINGS][NUMHACCEPTORS]
GT: 19.8 Gen: 17.82 diff

[13:13:51] SMILES Parse Error: syntax error while parsing: CC(C)CC(C(=O)c1ccc(F)cc1Cl)C(C)l
[13:13:51] SMILES Parse Error: Failed parsing SMILES 'CC(C)CC(C(=O)c1ccc(F)cc1Cl)C(C)l' for input: 'CC(C)CC(C(=O)c1ccc(F)cc1Cl)C(C)l'


GT: 23.0 <bos>[TPSA]23.00[/TPSA][NUMAROMATICRINGS]1[/NUMAROMATICRINGS][NHOHCOUNT]0[/NHOHCOUNT][NUMSATURATEDHETEROCYCLES]0[/NUMSATURATEDHETEROCYCLES][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][WEIGHT]235.08[/WEIGHT][RINGCOUNT]1[/RINGCOUNT][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][NUMALIPHATICRINGS]0[/NUMALIPHATICRINGS][NUMHDONORS]0[/NUMHDONORS][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][FORMULA]C12H15ClFNO[/FORMULA][IUPAC]1-(2-chloro-4-fluorophenyl)-2-methyl-3-propan-2-ylpropan-1-one[/IUPAC][NUMHETEROATOMS]4[/NUMHETEROATOMS][NUMROTATABLEBONDS]4[/NUMROTATABLEBONDS][START_SMILES]CC(C)CC(C(=O)c1ccc(F)cc1Cl)C(C)l[END_SMILES][NOCOUNT]1[/NOCOUNT][NUMAROMATICCARBOCYCLES]1[/NUMAROMATICCARBOCYCLES][CLOGP]4.33[/CLOGP][FRACTIONCSP3]0.50[/FRACTIONCSP3][NUMHACCEPTORS]
GT: 23.200000000000003 Gen: 20.310000000000002 diff: 2.89 Cc1ccccc1C(=O)C(C)N(C)CCS <bos>[TPSA]23.20[/TPSA][NUMAROMATICRINGS]1[/NUMAROMATICRINGS][NHOHCOUNT]0[/NHOHCOUNT][NUMSATURATEDHETEROCYCLES]0[/NUMSAT

[13:15:15] Explicit valence for atom # 11 C, 5, is greater than permitted


GT: 30.8 <bos>[TPSA]30.80[/TPSA][NUMAROMATICRINGS]1[/NUMAROMATICRINGS][NHOHCOUNT]0[/NHOHCOUNT][NUMSATURATEDHETEROCYCLES]0[/NUMSATURATEDHETEROCYCLES][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][WEIGHT]232.12[/WEIGHT][NUMALIPHATICRINGS]1[/NUMALIPHATICRINGS][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][NUMHDONORS]0[/NUMHDONORS][RINGCOUNT]2[/RINGCOUNT][NUMALIPHATICHETEROCYCLES]1[/NUMALIPHATICHETEROCYCLES][FORMULA]C13H16N2O2[/FORMULA][IUPAC]1-[(4-methoxyphenyl)methyl]-4,5-dihydroimidazol-2-one[/IUPAC][START_SMILES]COc1ccc(CN2CCN=C2=O)cc1[END_SMILES][QED]0.77[/QED][NUMAROMATICCARBOCYCLES]1[/NUMAROMATICCARBOCYCLES][NUMHETEROATOMS]4[/NUMHETEROATOMS][FRACTIONCSP3]0.31[/FRACTIONCSP3][NOCOUNT]4[/NOCOUNT][NUMROTATABLEBONDS]3[/NUMROTATABLEBONDS][CLOGP]1.86[/CLOGP][NUMHACCEPTORS]
GT: 31.0 Gen: 39.92 diff: 8.92 Cc1nc(N(C)C2CCCC2)sc1C#N <bos>[TPSA]31.00[/TPSA][NUMAROMATICRINGS]1[/NUMAROMATICRINGS][NHOHCOUNT]0[/NHOHCOUNT][NUMSATURATEDHETEROCYCLES]0[/NUMSATURATEDHETEROCYCLES][WEIGHT]232.1

[13:27:50] Can't kekulize mol.  Unkekulized atoms: 16 17 18


GT: 95.2 <bos>[TPSA]95.20[/TPSA][NUMAROMATICRINGS]2[/NUMAROMATICRINGS][NHOHCOUNT]2[/NHOHCOUNT][NUMSATURATEDHETEROCYCLES]0[/NUMSATURATEDHETEROCYCLES][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][RINGCOUNT]2[/RINGCOUNT][WEIGHT]332.12[/WEIGHT][NUMHDONORS]2[/NUMHDONORS][NUMAROMATICHETEROCYCLES]1[/NUMAROMATICHETEROCYCLES][NUMROTATABLEBONDS]6[/NUMROTATABLEBONDS][NUMALIPHATICRINGS]0[/NUMALIPHATICRINGS][FORMULA]C16H19N3O4[/FORMULA][IUPAC]N-[2-(2-methyl-6-nitroanilino)ethyl]-2-oxo-1-propan-2-ylpyrrole-3-carboxamide[/IUPAC][START_SMILES]Cc1cccc([N+](=O)[O-])c1NCCNC(=O)c1ccn(C(C)C)c1=O[END_SMILES][CLOGP]2.26[/CLOGP][NUMHETEROATOMS]7[/NUMHETEROATOMS][NUMAROMATICCARBOCYCLES]1[/NUMAROMATICCARBOCYCLES][NOCOUNT]7[/NOCOUNT][FRACTIONCSP3]0.31[/FRACTIONCSP3][NUMHACCEPTORS]
GT: 95.4 Gen: 150.05 diff: 54.65 COC(=O)c1nc2ccccc2c(=O)o1.Cc1nc(CC(=O)NC2=NCC(=O)CO2)cs1 <bos>[TPSA]95.40[/TPSA][NUMAROMATICRINGS]2[/NUMAROMATICRINGS][NHOHCOUNT]2[/NHOHCOUNT

In [14]:
ground_truths, gens, diffs = [],[],[]
invalids = 0
for i in np.arange(0,1, 0.01):
    prompt = f"<bos>[QED]{i:.2f}[/QED]"
    len_prompt = len(prompt)
    prompt = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(prompt.input_ids, do_sample=False, eos_token_id=20, max_length=300)
    out = tokenizer.batch_decode(out)[0]
    try:
        if out.find("[END_SMILES]")!=-1:
            smiles = out[out.find("[START_SMILES]") + len("[START_SMILES]"):out.find("[END_SMILES]")]
            # tpsa_score = AllChem.CalcTPSA(Chem.MolFromSmiles(smiles))
            qed_score = QED.qed(Chem.MolFromSmiles(smiles))
            diff = abs(i - qed_score)
            print("GT:", i, "Gen:", qed_score, "diff:", round(diff,2), smiles, out )
            ground_truths.append(i)
            gens.append(qed_score)
            diffs.append(diff)
        else:
            print(f"GT: {i} {out}")
    except:
        print(f"GT: {i} {out}")
        invalids += 1
        pass

GT: 0.0 <bos>[QED]0.00[/QED][TPSA]111.16[/TPSA][NUMAROMATICRINGS]10[/NUMAROMATICRINGS][WEIGHT]1166.46[/WEIGHT][NHOHCOUNT]0[/NHOHCOUNT][NUMHDONORS]0[/NUMHDONORS][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][RINGCOUNT]10[/RINGCOUNT][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][FORMULA]C76H70N2O6[/FORMULA][CLOGP]19.66[/CLOGP][START_SMILES]C=CC(=O)OCCCCCCOc1ccc(C(=O)Oc2ccc(OC(=O)c3ccc(OCCCCCCOC(=O)C=C)cc3)c(C=NN(CCCCCC)c3nc4ccccc4s3)c2)cc1[END_SMILES][NUMAROMATICHETEROCYCLES]1[/NUMAROMATICHETEROCYCLES][NOCOUNT]10[/NOCOUNT][NUMALIPHATICRINGS]0[/NUMALIPHATICRINGS][NUMAROMATICCARBOCYCLES]7[/NUMAROMATICCARBOCYCLES][FRACTIONCSP3]0.30[/FRACTIONCSP3][NUMHETEROATOMS]11[/NUMHETEROATOMS][NUMHACCEPTORS]
GT: 0.01 <bos>[QED]0.01[/QED][TPSA]236.95[/TPSA][NUMAROMATICRINGS]0[/NUMAROMATICRINGS][NHOHCOUNT]1[/NHOHCOUNT][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][NUMHDONORS]1[/NUMHDONORS][NUMROTATABLEBONDS]55[/NUMROTATABLEBONDS][PROPERTY]Other MS 1255.949 100[/PROPERTY][PROPERTY]Other

: 